# Advanced Python: Building Scalable Applications

### Day 6: Data Persistence, Web APIs and Profiling Code


In [3]:
import pickle
a = 10
b = [11, 22, 33, 4]
c = {"x": 100, "y": 200, "z": (34, 54, 56)}

with open("pickled.dat", "wb") as outs:
    pickle.dump(a, outs)
    pickle.dump(b, outs)
    pickle.dump(c, outs)

In [4]:
ins = open("pickled.dat", "rb")
ins

<_io.BufferedReader name='pickled.dat'>

In [7]:
pickle.load(ins)

{'x': 100, 'y': 200, 'z': (34, 54, 56)}

In [8]:
ins.close()

In [22]:
import shelve
# shelve module implements a "persistent" dictionary

d = shelve.open("shelved_data")
d

In [11]:
d["name"] = "Guido"

In [12]:
d["role"] = "Founder"

In [13]:
d["place"] = "San Francisco"
d

In [15]:
d["place"], d["role"]

('San Francisco', 'Founder')

In [16]:
"name" in d

True

In [17]:
for k, v in d.items():
    print(k, v)

name Guido
role Founder
place San Francisco


In [18]:
d.close()

In [23]:
d["name"]

'Raymond'

In [37]:
a = 1000_000_000_000_000_000_000_000_100
hash(a)

2292473209843970632

In [39]:
a = "Hello_world"
b = "Hello_world"
print(id(a), id(b))

2208434057200 2208434057200


In [26]:
a = "this is a test string"
b = "this is a test string"
print(id(a), id(b))
a == b
print(hash(a), hash(b))

2208433088640 2208432439040
-571275007076476873 -571275007076476873


In [ ]:
# In regular dictionaries - keys must be hashable object
# However, a shelve object expects a key to be a "str" object


In [65]:
d = { str(v): v*v for v in range(100) }
d["5"]

25

In [66]:
import sys
sys.getsizeof(d)

4696

In [67]:
import shelve
with shelve.open("squares") as s:
    s.update(d)

In [50]:
d = {}
print(sys.getsizeof(d))
d["name"] = "john"
d["role"] = "admin"
d["place"] = "bengaluru"
print(sys.getsizeof(d))
del d["role"]
del d["place"]
print(sys.getsizeof(d))
del d["name"]
print(d)
print(sys.getsizeof(d))




64
232
232
{}
232


In [71]:
import shelve

s = shelve.open("squares")
print(sys.getsizeof(s))
print(s["50"])
print(sys.getsizeof(s))


48
2500
48


#### Working with redis

In [72]:
pip install redis

Note: you may need to restart the kernel to use updated packages.


In [73]:
import redis

conn = redis.ConnectionPool(host="localhost", port=6379, db=0)
conn

ConnectionPool<Connection<host=localhost,port=6379,db=0>>

In [74]:
d = redis.Redis(connection_pool=conn)
d

Redis<ConnectionPool<Connection<host=localhost,port=6379,db=0>>>

In [75]:
d["name"] = "John"

In [76]:
d["name"]

b'Emily'

In [77]:
print(dir(d))

['RESPONSE_CALLBACKS', '__abstractmethods__', '__annotations__', '__class__', '__class_getitem__', '__contains__', '__del__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setitem__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_disconnect_raise', '_eval', '_evalsha', '_fcall', '_georadiusgeneric', '_geosearchgeneric', '_is_protocol', '_is_runtime_protocol', '_send_command_parse_response', '_zaggregate', '_zrange', 'acl_cat', 'acl_deluser', 'acl_dryrun', 'acl_genpass', 'acl_getuser', 'acl_help', 'acl_list', 'acl_load', 'acl_log', 'acl_log_reset', 'acl_save', 'acl_setuser', 'acl_users', 'acl_whoami', 'append', 'auth', 'bf', 'bgrewriteaof', 'bgsave',

In [80]:
"role" in d

True

In [82]:
d["place"] = "Noida"

In [83]:
d["place"]

b'Noida'

In [86]:
d.expire("place", 20)

True

In [89]:
d["place"]

KeyError: 'place'

In [1]:
# Redis simple publish-subscribe example
from redis import Redis
r = Redis() # connects to 'localhost:6379' and uses db=0
print(r)
ps = r.pubsub()
ps
print(ps)

Redis<ConnectionPool<Connection<host=localhost,port=6379,db=0>>>


In [94]:
def simple_subscriber(*args, **kwargs):
    print(f"*** got message: {args=}, {kwargs=}")

ps.subscribe(test_event=simple_subscriber)


In [3]:
def simple_subscriber(info):
    print(f"*** got message on channel {info['channel']}, message {info['data']}")
    for k, v in info.items():
        print(f"    {k} => {v}")


ps.subscribe(test_event=simple_subscriber)


In [ ]:
for m in ps.listen():
    print(m)

{'type': 'subscribe', 'pattern': None, 'channel': b'test_event', 'data': 1}
*** got message on channel b'test_event', message b'this is a test message'
    type => message
    pattern => None
    channel => b'test_event'
    data => b'this is a test message'
